## london 
[link](https://www.kaggle.com/c/data-science-london-scikit-learn/data)  
二元分類問題  

In [21]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#names=['',''....]
data = pd.read_csv('train.csv', header = None)
data_y = pd.read_csv('trainLabels.csv', header = None,names=['Label'])
y = [row for row in data_y.iloc[:,0]]
X = data.iloc[:,:].copy()
#y = data.iloc[:,-1]
#y =  data.pop('SalePrice').values
#data.head()
#y.head()
#X.head()
#y[:5]

### 確認是否有缺失

In [22]:
#X.info()
x_null_count = X.isnull().sum(axis = 0)
columns_name = X.columns.values
columns_has_nan_name = columns_name[x_null_count > 0]

print('Has %d columns Nan'%len(columns_has_nan_name))
x_null_count[x_null_count>0]

Has 0 columns Nan


Series([], dtype: int64)

In [23]:
class dataProcess:
    def __init__(self):
        self.option = {'aaa':0}
        self.caterogy_imputer_step = ('caterogy_imputer_step', SimpleImputer(strategy='constant',fill_value='MISSING')) 
        self.caterogy_encoder_step = ('caterogy_encoder_step', OneHotEncoder(sparse=False,handle_unknown='ignore')) 
        self.num_imputer_step = ('num_imputer_step', SimpleImputer(strategy='median'))
        self.num_scaler_step = ('num_scaler_step', StandardScaler())
        self.caterogy_pipe = Pipeline([self.caterogy_imputer_step, self.caterogy_encoder_step])
        self.num_pipe = Pipeline([self.num_imputer_step, self.num_scaler_step])
        pass
    def fit(self, datax, datay=None):
        #print('dataProcess call fit')
        kinds = np.array([dt.kind for dt in datax.dtypes])
        is_num = kinds != 'O'
        numeric_column_name = datax.columns.values[is_num]
        category_column_name = datax.columns.values[~is_num]
        self.ct_transformer = [('caterogy_transformer',self.caterogy_pipe, category_column_name),('num_transformer',self.num_pipe,numeric_column_name)]
        self.ct = ColumnTransformer(transformers=self.ct_transformer)        
    def transform(self, datax, datay=None):
        #print('dataProcess call transform')
        #try:
        #    if self.features < X.shape[1]:
        #        if Y is not None:
        #            self.ct.fit(X, Y)
        #        return self.ct.transform(datax)
        #except Exception as err:
        #    print('dataProcess.transform(): {}'.format(err))
        return self.ct.transform(datax)
    def fit_transform(self, datax, datay=None):
        #print('dataProcess call fit_transform')
        self.fit(datax)
        return self.ct.fit_transform(datax)
    def set_params(self,a,b):
        print(a)
        print(b)
        pass
    #def get_params(self):
    #    print("get_params")
    #    return [""]

###  regularization
- LogisticRegression:  
C: Inverse of regularization strength  
solver: liblinear,sag,newton-cg,lbfgs,saga  
multi_class: ovr,multinomial  
- Ridge  
alpha: Regularization strength  
- Lasso  
alpha: Constant that multiplies the L1 term
- SVC  
C: Penalty parameter C of the error term.  


In [107]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

#eval_set =  [(X_test, y_test)]
#model = XGBClassifier()
#model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="auc",
#          eval_set=eval_set, verbose=True)

ml_pipe = Pipeline([('dataProcess',dataProcess()), ('GradientBoostingClassifier', GradientBoostingClassifier(n_estimators=100,criterion='friedman_mse',  loss='deviance',learning_rate=0.17,max_depth=3, max_features=None, min_samples_leaf=1, min_samples_split=2))])
#ml_pipe = Pipeline([('dataProcess',dataProcess()), ('RandomForestClassifier', RandomForestClassifier(n_estimators=100,criterion='gini', max_depth=9, max_features='auto', min_samples_leaf=1, min_samples_split=2))])
#ml_pipe = Pipeline([('dataProcess',dataProcess()), ('logisticRegression', LogisticRegression(solver='liblinear',max_iter=1000,multi_class='ovr',C=1))])
#ml_pipe = Pipeline([('dataProcess',dataProcess()), ('dt', DecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1))])
#ml_pipe = Pipeline([('dataProcess',dataProcess()), ('svc', SVC(kernel='rbf', C=1,degree=3, gamma=1))])
# 1) call dataProcess fit_transform() --> Ridge()
ml_pipe.fit(X,y)
prediction = ml_pipe.predict(X)
print(np.sqrt(metrics.mean_squared_error(y_pred=prediction,y_true=y)))

#import matplotlib.pyplot as plt
#%matplotlib inline
#plt.figure(figsize=(20,10))
#x = np.arange(len(y))
#plt.plot(x,y,c='b')
#plt.plot(x,prediction,c='r')
#plt.show()

0.0


In [108]:
data2 = pd.read_csv('test.csv', header = None)
X2 = data2.iloc[:,:].copy()

prediction = ml_pipe.predict(data2)
prediction.shape

(9000,)

In [109]:
result_pd = pd.DataFrame(prediction,columns=['Solution'])
result_pd.index += 1
result_pd.to_csv('result.csv',index=True, index_label='Id')